In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.preprocessing import scale
from sklearn.ensemble import RandomForestRegressor 
import re
from sklearn.linear_model import LinearRegression
import sys
from sklearn.linear_model import LogisticRegression
sys.path.append("./")
from poor_model import *
from rich_model import *
from connected_model import *
import warnings
from sklearn.cluster import KMeans
warnings.filterwarnings("ignore",category=FutureWarning)
train = pd.read_csv("./train.csv")
train = train.dropna(subset=["賃料"])
train = train.drop(20926)
train = train.drop(20231)
train = train.drop(20427)
train = train.drop(5775)
train_x = train.drop("賃料",axis=1)
train_y = train["賃料"]
test = pd.read_csv("./test.csv")
x_small,x_valid,y_small,y_valid = train_test_split(train_x, train_y, test_size=0.25, random_state=7777)
import lightgbm as lgbm
train_rich = train.query("賃料>150000")
rich_x = train_rich.drop("賃料",axis=1)
rich_y = train_rich["賃料"]
train_poor = train.query("賃料<=150000")
poor_x = train_poor.drop("賃料",axis=1)
poor_y = train_poor["賃料"]

/Users/kakeru/PycharmProjects/main_env/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
comment = """envを落として、安定版"""
check_rich (comment,rich_x,rich_y)

[15:56:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:56:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:56:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:56:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[46086.2082318  35521.68590928 48992.07283317 30192.34788562]
40198.07871496792
[15:56:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [4]:
comment = """poor側の精度検証"""
check_poor(comment,poor_x,poor_y)

[15:00:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:01:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:02:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:02:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[7733.39777641 7793.89219298 7690.03201154 7610.92459196]
[15:03:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [2]:
check_splitter(x_small,x_valid,y_small,y_valid)

0.9495360366086183


In [ ]:
comment = """合体版のスコア。"""
check_specks(comment,train_x,train_y)

In [2]:
each_models(train_x,train_y)

[12:48:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
8518.094245091275
18771.755124734744
14009.61826113935


In [3]:
commit(easy_model(),train_x,train_y,test,"submit_10")

[11:07:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [3]:
m = pre_checker(train_x,train_y)
hoge = m.predict(train)

In [5]:
easy = easy_model()
easy.fit(train_x,train_y)
pred = easy.predict(train_x)
score = mean_squared_error(pred,train_y)
print(np.sqrt(score))
diff = abs(train_y.values-pred)
diff = pd.DataFrame(diff)
diff.columns = ["diff"]
diff.index = train_x.index
diff = pd.concat([train_x,train_y,diff],axis=1)

[14:21:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
8141.015612451783


In [6]:
diff = diff.sort_values('diff',ascending=False)
diff.head()

,id,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間,賃料,diff
21737,21738,東京都豊島区上池袋１丁目14-1,山手線\t池袋駅\t徒歩12分\t\t東武東上線\t北池袋駅\t徒歩11分\t\t山手線\t...,1R,26年7ヶ月,NaN,18.56m2,3階／3階建,専用バス／\t専用トイレ／\tシャワー,電気コンロ／\tコンロ1口／\t給湯,NaN,エアコン付\t出窓／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水／\t都市ガス,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【スーパー】 265m\t【スーパー】 273m\t【コンビニ】 203m\t【コンビニ】 ...,鉄骨造,2年間,157000,71460.726562
13785,13786,東京都北区田端１丁目,山手線\t田端駅\t徒歩6分\t\t京浜東北線\t西日暮里駅\t徒歩10分\t\t山手線\t...,1K,8年1ヶ月,NaN,22.54m2,10階／11階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t独立キッチン,インターネット対応／\t光ファイバー／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t24時間換気システム\...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 148m\t【スーパー】 428m\t【コンビニ】 363m\t【コンビニ】 ...,RC（鉄筋コンクリート）,NaN,174500,71459.437500
23316,23317,東京都世田谷区代田５丁目,京王井の頭線\t下北沢駅\t徒歩2分\t\t京王井の頭線\t新代田駅\t徒歩8分\t\t小田...,2DK,40年6ヶ月,東,44.4m2,2階／3階建,専用バス／\t専用トイレ／\tバス・トイレ別,ガスコンロ／\tコンロ3口,インターネット対応,エアコン付\t公営水道／\t下水／\t都市ガス,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【スーパー】 39m\t【スーパー】 39m\t【スーパー】 122m\t【コンビニ】 60...,RC（鉄筋コンクリート）,2年間,210000,71205.312500
24184,24185,東京都大田区中馬込１丁目,都営浅草線\t馬込駅\t徒歩5分\t\t都営浅草線\t西馬込駅\t徒歩11分\t\t東急大井...,1K,18年1ヶ月,南,19.08m2,4階／5階建,専用バス／\t専用トイレ／\tバス・トイレ別,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t高速インターネット／\t光ファイバー／\tCATV／\tCSアンテナ...,冷房／\tエアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t24時間換気...,"駐車場\t近隣\t30,000円\t距離400m",【スーパー】 632m\t【コンビニ】 201m\t【コンビニ】 505m\t【コンビニ】 ...,RC（鉄筋コンクリート）,NaN,154000,71012.328125
5311,5312,東京都練馬区下石神井６丁目28-2,西武池袋線\t石神井公園駅\t徒歩13分\t\t西武池袋線\t練馬高野台駅\t徒歩19分\t...,2LDK,7年10ヶ月,南,67.5m2,2階建,専用バス／\t専用トイレ／\tシャワー／\t追焚機能／\t温水洗浄便座／\t脱衣所,IHコンロ\t／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV,エアコン付\tフローリング／\t3面採光／\t室内洗濯機置場／\t敷地内ごみ置き場\t公営水...,"駐車場\t空有\t10,000円(税込)\t(屋根付)\t駐輪場\t無\tバイク置き場\t無",【スーパー】 1040m\t【スーパー】 1040m,鉄骨造,2年間,215000,67625.640625


In [7]:
diff[:500].to_csv("human_check.csv")

In [59]:
grid_params2={
    "xgb__n_estimators":[100,120,140],
    "xgb__max_depth":[7,8,9],
    "xgb__min_child_weight":[0,5,15],
    "xgb__learning_rate":[0.1,0.2],
    "xgb__random_state":[7777]
}
GCV2 = GridSearchCV(
    Pipeline(steps=rich_step),
    grid_params2,
    cv = 3,
    scoring="neg_mean_squared_error",
    verbose=3
)
GCV2.fit(temp_x_rich,temp_y_rich)

Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777 
[17:05:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777, score=-20533275032.002, total=   0.6s
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777 
[17:05:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777, score=-15907774518.319, total=   0.6s
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777 
[17:05:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.3s remaining:    0.0s


[CV]  xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=100, xgb__random_state=7777, score=-9381239329.321, total=   0.6s
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=120, xgb__random_state=7777 
[17:05:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=120, xgb__random_state=7777, score=-20454789224.898, total=   0.7s
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=120, xgb__random_state=7777 
[17:05:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[CV]  xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_child_weight=0, xgb__n_estimators=120, xgb__random_state=7777, score=-15826305572.156, total=   0.9s
[CV] xgb__learning_rate=0.1, xgb__max_depth=7, xgb__min_c

[Parallel(n_jobs=1)]: Done 162 out of 162 | elapsed:  2.2min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('drop_id',
                                        <my_class.drop_id object at 0x134b3b690>),
                                       ('parse_area',
                                        <my_class.parse_area_size object at 0x136792550>),
                                       ('parse_room',
                                        <my_class.parse_rooms object at 0x136792a50>),
                                       ('parse_old',
                                        <my_class.parse_how_old object at 0x136792990>),
                                       ('str_label',
                                        <my_class.structure_lab...
                                                     seed=None, silent=None,
                                                     subsample=1,
                                                     verbosity=1))],
         

In [810]:
crazy_pred = crazy["賃料"].median()
print(crazy_pred)
pred = pd.DataFrame([crazy_pred for i in range(len(crazy.index))])
print(np.sqrt(mean_squared_error(crazy["賃料"],pred)))

650000.0
301848.99189789325
